# Q Function
* used to approximate the reward based on a state
* Q(s,a) calculates the expected future value from state **s** and action **a**
* in DQN, we use a **neural network to approximate the reward**

# Classes
* Environment
* Agent
* Runner

## Environment

In [1]:
class Action:
    def __init__(self, act, days, percentage):
        self.act = act
        self.days = days
        self.percentage = percentage

In [2]:
import numpy as np
import pandas as pd
import pandas_datareader as pdr
import datetime

BUY = 'buy'
SELL = 'sell'
SKIP = 'skip'

class Environment:
    
    min_days_to_hold = 5
    max_days_to_hold = 15
    max_drawdowns = 3
    
    def __init__(self, 
                 ticker, 
                 initial_deposit = 100000,
                 from_date = datetime.datetime(2007, 1, 1), 
                 to_date = datetime.datetime(2017, 1, 1),
                 window = 50):
        self.initial_deposit = initial_deposit
        self.window = window
        self.data = pdr.get_data_google(ticker, from_date, to_date)
        self.pct_data = self.data.pct_change().fillna(0)
        self.data_length = len(self.data)
        
        actions = np.array([BUY, SELL, SKIP])
        days_to_holds = np.arange(Environment.min_days_to_hold, 
                                  Environment.max_days_to_hold + 1,
                                  2)
        
        self.action_space = [Action(act, days, 3) for act in actions for days in days_to_holds]
        self.reset()
        
    def reset(self):
        self.deposit = self.initial_deposit
        self.current_index = self.window
        self.actions = {}
        self.drawdowns = 0
        
        return self.state()
    
    def score(self):
        return self.deposit
    
    def enough_data_provided(self):
        return self.current_index + Environment.max_days_to_hold <= self.data_length
    
    def state(self):
        return self.pct_data.iloc[self.current_index - self.window:self.current_index]['Close']
    
    # def price_state(self):
    #    return self.data.iloc[self.current_index - self.window:self.current_index]['Close']
        
    def state_size(self):
        return self.window
    
    def action_size(self):
        return len(self.action_space)
        
    def step(self, action_idx: int):
        action = self.action_space[action_idx]
        #print('\t=> current action is: {} at {}'.format(action, self.data.index[self.current_index]))
        
        df = self.data.iloc[self.current_index: self.current_index + action.days]
        on_date = df.index[0]
        first_day_price = df.iloc[0]['Close']
        last_day_price = df.iloc[-1]['Close']
        
        if action.act == BUY:
            reward = last_day_price - first_day_price
        elif action.act == SELL:
            reward = first_day_price - last_day_price
        elif action.act == SKIP:
            reward = 0
            
        self.actions[on_date] = (action, reward)
        
        self.current_index += action.days
        self.deposit += reward * (self.deposit*action.percentage/100)
        
        if reward < 0:
            self.drawdowns += 1
        else:
            self.drawdowns = 0
        
        next_state = self.state()
        done = self.drawdowns > Environment.max_drawdowns
        _ = None
        return next_state, reward, done, _ 

## Agent

In [3]:
import random
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from keras.losses import mean_squared_error

class Agent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95    # discount rate
        self.epsilon = 1.0  # exploration rate
        self.epsilon_min = 0.05
        self.epsilon_decay = 0.999
        self.learning_rate = 0.001
        self.model = self._build_model()
    
    def _build_model(self):
        # Neural Net for Deep-Q learning Model
        model = Sequential()
        model.add(Dense(48, input_dim=self.state_size, activation='relu'))
        model.add(Dense(36, activation='relu'))
        model.add(Dense(24, activation='relu'))
        model.add(Dense(self.action_size, activation='linear'))
        model.compile(loss=mean_squared_error,
                      optimizer=Adam(lr=self.learning_rate))
        return model

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        act_values = self.model.predict(state)
        return np.argmax(act_values[0])  # returns action

    def replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            target = reward
            if not done:
                target = reward + self.gamma * np.amax(self.model.predict(next_state)[0])
            target_f = self.model.predict(state)
            target_f[0][action] = target
            self.model.fit(state, target_f, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    def load(self, name):
        self.model.load_weights(name)

    def save(self, name):
        self.model.save_weights(name)

Using TensorFlow backend.


## Runner

In [4]:
env = Environment('AAPL')
state_size = env.state_size()
action_size = env.action_size()
print('Action size: {}, state size: {}'.format(action_size, state_size))

Action size: 18, state size: 50


In [ ]:
agent = Agent(state_size, action_size)
EPISODES = 50
batch_size = 32

In [ ]:
max_reward = None
for e in range(EPISODES):
    print('==> EPISODE {}'.format(e))
    state = env.reset()
    state = state.values.reshape([1, state_size])
    while env.enough_data_provided():
        action_idx = agent.act(state)
        next_state, reward, done, _ = env.step(action_idx) # build these parameters into the NN model
        # reward = reward if not done else -10
        next_state = next_state.values.reshape([1, state_size])
        agent.remember(state, action_idx, reward, next_state, done)
        state = next_state
        
        if done:
            break
        elif len(agent.memory) > batch_size:
            agent.replay(batch_size)
            print("episode: {}/{}, score: {}, e: {:.2}".format(e, EPISODES, env.score(), agent.epsilon))
    
    if max_reward is None or max_reward[0] < env.score():
        max_reward = (env.score(), env.actions)

==> EPISODE 0
episode: 0/50, score: 100592.29045986259, e: 1.0
episode: 0/50, score: 95160.30677503001, e: 1.0
episode: 0/50, score: 92876.45941242929, e: 1.0
episode: 0/50, score: 97752.47353158183, e: 1.0
episode: 0/50, score: 115875.7821243371, e: 1.0
episode: 0/50, score: 115736.7311857879, e: 0.99
episode: 0/50, score: 115736.7311857879, e: 0.99
episode: 0/50, score: 107993.94386945869, e: 0.99
episode: 0/50, score: 107993.94386945869, e: 0.99
episode: 0/50, score: 109387.06574537471, e: 0.99
episode: 0/50, score: 108140.05319587744, e: 0.99
episode: 0/50, score: 108140.05319587744, e: 0.99
episode: 0/50, score: 112876.58752585687, e: 0.99
episode: 0/50, score: 118836.47134722212, e: 0.99
episode: 0/50, score: 123684.99937818879, e: 0.99
episode: 0/50, score: 129213.71885039382, e: 0.98
episode: 0/50, score: 126422.7025232253, e: 0.98
episode: 0/50, score: 128356.96987183065, e: 0.98
episode: 0/50, score: 125622.96641356066, e: 0.98
episode: 0/50, score: 127205.81579037153, e: 0.9

episode: 1/50, score: 112888.82877379819, e: 0.85
episode: 1/50, score: 112888.82877379819, e: 0.84
episode: 1/50, score: 112888.82877379819, e: 0.84
episode: 1/50, score: 114717.62779993372, e: 0.84
episode: 1/50, score: 114717.62779993372, e: 0.84
episode: 1/50, score: 107765.73955525774, e: 0.84
episode: 1/50, score: 107765.73955525774, e: 0.84
episode: 1/50, score: 107765.73955525774, e: 0.84
episode: 1/50, score: 107765.73955525774, e: 0.84
episode: 1/50, score: 108509.32315818901, e: 0.84
episode: 1/50, score: 103886.82599165017, e: 0.84
episode: 1/50, score: 103824.49389605518, e: 0.84
episode: 1/50, score: 105008.09312647021, e: 0.84
episode: 1/50, score: 96848.96429054347, e: 0.83
episode: 1/50, score: 100044.98011213141, e: 0.83
episode: 1/50, score: 100285.08806440052, e: 0.83
episode: 1/50, score: 100375.34464365848, e: 0.83
episode: 1/50, score: 100375.34464365848, e: 0.83
episode: 1/50, score: 100375.34464365848, e: 0.83
episode: 1/50, score: 108505.7475597948, e: 0.83
ep

episode: 1/50, score: 229987.82322088233, e: 0.72
episode: 1/50, score: 206322.0762114535, e: 0.72
episode: 1/50, score: 218515.7109155504, e: 0.71
episode: 1/50, score: 218515.7109155504, e: 0.71
episode: 1/50, score: 200291.50062519347, e: 0.71
episode: 1/50, score: 156427.66198827612, e: 0.71
episode: 1/50, score: 162716.05400020484, e: 0.71
episode: 1/50, score: 158273.90572599927, e: 0.71
episode: 1/50, score: 158273.90572599927, e: 0.71
episode: 1/50, score: 158273.90572599927, e: 0.71
episode: 1/50, score: 117581.68456384489, e: 0.71
episode: 1/50, score: 122625.9388316338, e: 0.71
episode: 1/50, score: 122625.9388316338, e: 0.71
episode: 1/50, score: 122625.9388316338, e: 0.71
episode: 1/50, score: 117365.28605575673, e: 0.71
episode: 1/50, score: 118421.57363025853, e: 0.71
episode: 1/50, score: 96537.26682338672, e: 0.7
episode: 1/50, score: 107542.5152412528, e: 0.7
episode: 1/50, score: 107542.5152412528, e: 0.7
episode: 1/50, score: 123157.68845428272, e: 0.7
==> EPISODE 2

episode: 3/50, score: 101560.0, e: 0.61
episode: 3/50, score: 100067.068, e: 0.61
episode: 3/50, score: 107061.7560532, e: 0.6
episode: 3/50, score: 107382.94132135961, e: 0.6
episode: 3/50, score: 112505.10762238846, e: 0.6
episode: 3/50, score: 113517.65359098995, e: 0.6
episode: 3/50, score: 110861.34049696078, e: 0.6
episode: 3/50, score: 115850.10081932401, e: 0.6
episode: 3/50, score: 121480.41571914316, e: 0.6
episode: 3/50, score: 121480.41571914316, e: 0.6
episode: 3/50, score: 121480.41571914316, e: 0.6
episode: 3/50, score: 119913.31835636622, e: 0.6
episode: 3/50, score: 117503.06065740326, e: 0.6
episode: 3/50, score: 127902.08152558345, e: 0.6
episode: 3/50, score: 127902.08152558345, e: 0.6
episode: 3/50, score: 122069.74660801684, e: 0.6
episode: 3/50, score: 118297.79143782913, e: 0.6
episode: 3/50, score: 118297.79143782913, e: 0.6
episode: 3/50, score: 115990.98450479146, e: 0.59
episode: 3/50, score: 95634.56672420056, e: 0.59
episode: 3/50, score: 98159.31928571947

episode: 4/50, score: 151033.66775617228, e: 0.51
episode: 4/50, score: 151033.66775617228, e: 0.51
episode: 4/50, score: 151033.66775617228, e: 0.51
episode: 4/50, score: 165215.72915847687, e: 0.51
episode: 4/50, score: 189205.05303228772, e: 0.51
episode: 4/50, score: 189205.05303228772, e: 0.51
episode: 4/50, score: 189205.05303228772, e: 0.51
episode: 4/50, score: 206914.64599610984, e: 0.51
episode: 4/50, score: 206914.64599610984, e: 0.51
episode: 4/50, score: 192823.7586037748, e: 0.51
episode: 4/50, score: 192823.7586037748, e: 0.51
episode: 4/50, score: 192823.7586037748, e: 0.51
episode: 4/50, score: 238175.90662738262, e: 0.51
episode: 4/50, score: 282119.3614001346, e: 0.51
episode: 4/50, score: 215680.25179040298, e: 0.5
episode: 4/50, score: 204809.96710016672, e: 0.5
episode: 4/50, score: 170770.55056811904, e: 0.5
==> EPISODE 5
episode: 5/50, score: 101410.0, e: 0.5
episode: 5/50, score: 101410.0, e: 0.5
episode: 5/50, score: 103843.84, e: 0.5
episode: 5/50, score: 107

...

episode: 26/50, score: 165894.28588550977, e: 0.061
episode: 26/50, score: 165894.28588550977, e: 0.061
episode: 26/50, score: 158130.43330606792, e: 0.061
episode: 26/50, score: 155853.35506646053, e: 0.061
episode: 26/50, score: 159921.12763369514, e: 0.061
episode: 26/50, score: 159921.12763369514, e: 0.061
episode: 26/50, score: 174985.69785678922, e: 0.06
episode: 26/50, score: 174985.69785678922, e: 0.06
episode: 26/50, score: 195301.53737796246, e: 0.06
episode: 26/50, score: 196180.3942961633, e: 0.06
episode: 26/50, score: 196180.3942961633, e: 0.06
episode: 26/50, score: 200417.8908129604, e: 0.06
episode: 26/50, score: 200417.8908129604, e: 0.06
episode: 26/50, score: 191399.08572637718, e: 0.06
episode: 26/50, score: 199380.42760116712, e: 0.06
episode: 26/50, score: 226775.29835356746, e: 0.06
episode: 26/50, score: 184186.8973227675, e: 0.06
episode: 26/50, score: 185402.53084509776, e: 0.06
episode: 26/50, score: 189907.81234463362, e: 0.06
episode: 26/50, score: 169796.

episode: 27/50, score: 214228.93338496218, e: 0.052
episode: 27/50, score: 211979.52958442006, e: 0.052
episode: 27/50, score: 211979.52958442006, e: 0.052
episode: 27/50, score: 223935.17505298133, e: 0.052
episode: 27/50, score: 223935.17505298133, e: 0.052
episode: 27/50, score: 198003.4817818461, e: 0.052
episode: 27/50, score: 198003.4817818461, e: 0.052
episode: 27/50, score: 198003.4817818461, e: 0.051
episode: 27/50, score: 198003.4817818461, e: 0.051
episode: 27/50, score: 184816.44989517512, e: 0.051
episode: 27/50, score: 187311.47196876, e: 0.051
episode: 27/50, score: 197875.83898779805, e: 0.051
episode: 27/50, score: 193601.72086566163, e: 0.051
episode: 27/50, score: 169265.98455284798, e: 0.051
episode: 27/50, score: 168351.9482362626, e: 0.051
==> EPISODE 28
episode: 28/50, score: 98440.0, e: 0.051
episode: 28/50, score: 98912.512, e: 0.051
episode: 28/50, score: 98912.512, e: 0.051
episode: 28/50, score: 98912.512, e: 0.051
episode: 28/50, score: 93838.30013440001, e

episode: 30/50, score: 318269.69582263, e: 0.05
episode: 30/50, score: 388734.60647776024, e: 0.05
episode: 30/50, score: 356080.8995336284, e: 0.05
episode: 30/50, score: 324674.56419476244, e: 0.05
episode: 30/50, score: 340064.13853759423, e: 0.05
episode: 30/50, score: 340064.13853759423, e: 0.05
episode: 30/50, score: 383014.2392348924, e: 0.05
episode: 30/50, score: 383014.2392348924, e: 0.05
episode: 30/50, score: 370834.3864272228, e: 0.05
episode: 30/50, score: 370834.3864272228, e: 0.05
episode: 30/50, score: 370834.3864272228, e: 0.05
episode: 30/50, score: 370834.3864272228, e: 0.05
episode: 30/50, score: 370834.3864272228, e: 0.05
episode: 30/50, score: 394196.9527721379, e: 0.05
episode: 30/50, score: 460540.29992368864, e: 0.05
episode: 30/50, score: 460540.29992368864, e: 0.05
episode: 30/50, score: 387176.230145845, e: 0.05
episode: 30/50, score: 464069.42945280985, e: 0.05
episode: 30/50, score: 464069.42945280985, e: 0.05
episode: 30/50, score: 427732.79312665487, e:

episode: 31/50, score: 83819.11520028199, e: 0.05
episode: 31/50, score: 90155.8403094233, e: 0.05
episode: 31/50, score: 79391.23297647816, e: 0.05
episode: 31/50, score: 64672.0983826391, e: 0.05
episode: 31/50, score: 64672.0983826391, e: 0.05
episode: 31/50, score: 64672.0983826391, e: 0.05
episode: 31/50, score: 63139.369650970555, e: 0.05
episode: 31/50, score: 63139.369650970555, e: 0.05
episode: 31/50, score: 63139.369650970555, e: 0.05
episode: 31/50, score: 63139.369650970555, e: 0.05
episode: 31/50, score: 63139.369650970555, e: 0.05
episode: 31/50, score: 63271.96232723759, e: 0.05
episode: 31/50, score: 61089.079626947896, e: 0.05
episode: 31/50, score: 64442.870098467334, e: 0.05
episode: 31/50, score: 68425.43947055262, e: 0.05
episode: 31/50, score: 68199.6355202998, e: 0.05
episode: 31/50, score: 71432.298243962, e: 0.05
episode: 31/50, score: 69932.21998083878, e: 0.05
episode: 31/50, score: 69932.21998083878, e: 0.05
episode: 31/50, score: 69932.21998083878, e: 0.05


episode: 32/50, score: 268618.75144509703, e: 0.05
episode: 32/50, score: 284897.04778267, e: 0.05


In [ ]:
signals = np.array([[], [], []]).T
signals.shape

In [ ]:
max_reward[0]

In [ ]:
for key, value in max_reward[1].items():
    print('Date: {} => Action: {} => Reward: {}'.format(key, value[0].act, value[1]))
    signals = np.concatenate((signals, np.array([[key, value[0].act, env.data.loc[key]['Close']]])), axis = 0)

In [ ]:
signals.shape

In [ ]:
signals_df = pd.DataFrame(signals)

In [ ]:
signals_df.set_index(keys = [0], inplace=True)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
fig = plt.figure(figsize=(20,20))

ax1 = fig.add_subplot(211,  ylabel='Price in $')
ax2 = fig.add_subplot(212,  ylabel='Signals')

# Plot the closing price
env.data['Close'].plot(ax=ax1, color='r', lw=2.)
env.data['Close'].plot(ax=ax2, color='r', lw=2.)

ax2.plot(signals_df.loc[signals_df[1] == 'skip'].index, 
         signals_df.loc[signals_df[1] == 'skip'][2],
         'o', markersize=10, color='b')

ax2.plot(signals_df.loc[signals_df[1] == 'sell'].index, 
         signals_df.loc[signals_df[1] == 'sell'][2],
         'v', markersize=10, color='k')

ax2.plot(signals_df.loc[signals_df[1] == 'buy'].index, 
         signals_df.loc[signals_df[1] == 'buy'][2],
         '^', markersize=10, color='m')
       
# Show the plot
plt.show()

In [ ]:
agent.save('best_model.h5')